In [1]:
#Preprocessing Pipeline
import pandas as pd
df=pd.read_csv("/content/train.csv")

In [2]:
df=df.drop(["PassengerId","Name","Ticket"],axis=1)
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,7.2500,NaN,S
1,1,1,female,38.0,1,0,71.2833,C85,C
2,1,3,female,26.0,0,0,7.9250,NaN,S
3,1,1,female,35.0,1,0,53.1000,C123,S
4,0,3,male,35.0,0,0,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,NaN,S
887,1,1,female,19.0,0,0,30.0000,B42,S
888,0,3,female,NaN,1,2,23.4500,NaN,S
889,1,1,male,26.0,0,0,30.0000,C148,C


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Cabin     204 non-null    object 
 8   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(3)
memory usage: 62.8+ KB


In [4]:
df["Age"]=df["Age"].fillna(df["Age"].median())

In [5]:
df["Embarked"]=df["Embarked"].fillna(df["Embarked"].mode())

In [6]:
df["Sex"]=df["Sex"].map({"male":0,"female":1})

In [7]:
df["Sex"].value_counts(dropna=False)

,count
Sex,
0,577
1,314


In [8]:
df=pd.get_dummies(df,columns=["Embarked"],drop_first=True)

In [9]:
df.columns

Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin',
       'Embarked_Q', 'Embarked_S'],
      dtype='object')

In [10]:
df=df.drop(["Cabin"],axis=1)

In [11]:
df.isnull().sum()
df.dtypes

,0
Survived,int64
Pclass,int64
Sex,int64
Age,float64
SibSp,int64
Parch,int64
Fare,float64
Embarked_Q,bool
Embarked_S,bool


In [12]:
#Train Logistic Regression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
x=df.drop(["Survived"],axis=1)
y=df["Survived"]
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
lr=LogisticRegression(max_iter=1000)
lr.fit(x_train,y_train)
train_acc_lr=accuracy_score(y_train,lr.predict(x_train))
test_acc_lr=accuracy_score(y_test,lr.predict(x_test))
train_acc_lr,test_acc_lr

(0.800561797752809, 0.8100558659217877)

In [13]:
from sklearn.tree import DecisionTreeClassifier
dt=DecisionTreeClassifier(random_state=42)
dt.fit(x_train,y_train)
train_acc_dt=accuracy_score(y_train,dt.predict(x_train))
test_acc_dt=accuracy_score(y_test,dt.predict(x_test))
train_acc_dt,test_acc_dt

(0.9789325842696629, 0.7877094972067039)

Why Decision Trees overfit more

Decision Trees split the data repeatedly to perfectly classify training samples.

With no depth restriction, they can memorize training data, including noise.

This leads to:

Very high training accuracy

Lower test accuracy → poor generalization

Overfitting = high train acc, low test acc

In [14]:
#Simple Control
dt_limited=DecisionTreeClassifier(max_depth=3,random_state=42)
dt_limited.fit(x_train,y_train)
train_acc_dt_1=accuracy_score(y_train,dt_limited.predict(x_train))
test_acc_dt_1=accuracy_score(y_test,dt_limited.predict(x_test))
train_acc_dt_1,test_acc_dt_1

(0.8342696629213483, 0.7988826815642458)

Logistic Regression

Is a linear model

Learns global patterns

Has lower variance

Cannot memorize individual samples easily

Generalizes better on unseen data

Decision Tree

Is a non-linear model

Learns local rules

Has high variance

Easily memorizes data

Needs constraints (like max_depth)

Logistic Regression
Train acc is ~0.80056
Test acc is ~0.81006
Decision Tree
Train acc is ~0.978933
Test acc is ~0.787709

Decision Tree (depth=3)
Train acc is ~0.834270
Test acc is ~0.798883

Day 11 Summary:
Learned the difference between training and testing accuracy.
Understood how overfitting appears as high train accuracy but low test accuracy.
Observed that Decision Trees overfit without depth control.
Learned that limiting model complexity improves generalization.
Compared Logistic Regression and Decision Tree models and selected the model with better generalization.

In [15]:
#Scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(x_train)
X_test_scaled = scaler.transform(x_test)

In [16]:
lr_scaled = LogisticRegression(max_iter=1000)
lr_scaled.fit(X_train_scaled, y_train)

train_acc = accuracy_score(y_train, lr_scaled.predict(X_train_scaled))
test_acc = accuracy_score(y_test, lr_scaled.predict(X_test_scaled))

train_acc, test_acc


(0.800561797752809, 0.8100558659217877)

Did scaling change accuracy?
No
Why scaling helps Logistic Regression?
Because it helps to have the same priority for features with high and low values
Does Decision Tree need scaling? (Yes / No)
No

Day 12 Summary:
Tried feature scaling using StandardScaler.
Observed its effect on Logistic Regression performance.
Understood that scaling helps distance-based and linear models but not Decision Trees.

# **Day 13 - Random Forest and Ensemble Learning**

In [17]:
#Train Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf=RandomForestClassifier(random_state=42)
rf.fit(x_train,y_train)
train_acc_rf = accuracy_score(y_train, rf.predict(x_train))
test_acc_rf=accuracy_score(y_test,rf.predict(x_test))
train_acc_rf,test_acc_rf

(0.9789325842696629, 0.7988826815642458)

In [18]:
#Decision Tree
dt_limited=DecisionTreeClassifier(max_depth=3,random_state=42)
dt_limited.fit(x_train,y_train)
train_acc_dt_1=accuracy_score(y_train,dt_limited.predict(x_train))
test_acc_dt_1=accuracy_score(y_test,dt_limited.predict(x_test))
train_acc_dt_1,test_acc_dt_1

(0.8342696629213483, 0.7988826815642458)

Train accuracy
0.83427
Test accuracy
0.79888

1)What problem of Decision Trees does Random Forest solve?
Random Forest addresses the overfitting problem of single decision trees by reducing variance through ensemble learning
2)What does “many trees” help with?
Using many trees trained on different subsets of data and features helps average out errors.
3)Why does Random Forest generalize better?
Although Random Forest generally improves generalization, in this case the model still overfits as indicated by high train accuracy and lower test accuracy.

Logistic Regression
Train Acc
0.800561797752809
Test Acc
0.8100558659217877
Overfitting
No
Decision Tree (depth=3)
Train Acc
0.8342696629213483
Test Acc
0.7988826815642458
Overfitting
No
Random Forest
Train Acc
0.9789325842696629
Test Acc
0.7988826815642458
Overfitting
Yes

In [19]:
rf.feature_importances_
x.columns

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked_Q',
       'Embarked_S'],
      dtype='object')

Which feature has the highest importance?
Pclass
Does it make sense logically for Titanic?
Yes,Pclass has the highest feature importance, which makes logical sense because passenger class strongly influenced survival chances on the Titanic.

# DAY 14  Cross-Validation & Model Reliability

1)Why train/test accuracy can change with different splits
beacause the train and test data ,class distribution + sample difficulty changes when we splits the data differently

2)Why relying on one split can be misleading
(the model might look good by chance on one split.)
One split can give optimistic or pessimistic accuracy

Cross-validation reduces this randomness

In [20]:
from sklearn.model_selection import cross_val_score
cv_scores_lr = cross_val_score(LogisticRegression(max_iter=1000),x,y,cv=5,scoring="accuracy")
cv_scores_lr,cv_scores_lr.mean()

(array([0.7877095 , 0.78651685, 0.78651685, 0.76966292, 0.8258427 ]),
 np.float64(0.7912497646098802))

What are the 5 scores?
0.7877095 , 0.78651685, 0.78651685, 0.76966292, 0.8258427


What does the mean accuracy represent?
Mean accuracy means calculating the mean for accuracys after different splits
0.7912497646098802

In [21]:
cv_scores_rf=cross_val_score(RandomForestClassifier(random_state=42),x,y,cv=5,scoring="accuracy")
cv_scores_rf,cv_scores_rf.mean()

(array([0.7877095 , 0.79775281, 0.86516854, 0.78089888, 0.80898876]),
 np.float64(0.8081036971941498))

| Model               | CV Mean Accuracy | Stability |
| ------------------- | ---------------- | --------- |
| Logistic Regression |   0.7912497646098802               |  More stable        |
| Random Forest       | 0.8081036971941498                 | Less stable         |


Does CV guarantee best model?
No,not sure
Does higher CV mean always mean better real-world performance?
Not sure it depends on the data and splits

Day-14 Summary:
Learned why train/test accuracy varies across different splits. Understood why relying on a single split can be misleading. Implemented 5-fold cross-validation to evaluate models more reliably. Compared Logistic Regression and Random Forest using CV mean accuracy and stability. Learned that higher CV accuracy does not always guarantee better real-world performance.